# Looking for duplicates on flood control projects of the Philippines

This notebook uses data from the Department of Budget and Management, specifically the list of DPWH projects included in the National Expenditure Program for 2026, as well as the list of projects that can be downloaded from DIME (Digital Information for Monitoring and Evaluation) website.

In [ ]:
import pandas as pd
import numpy as np
import re
import fuzzy_pandas as fpd
import pdfplumber

## PDF plumber: to convert the list of DPWH projects to CSV

<b>Before doing this:</b> We stripped out the pages containing the flood control projects from the list to manage the volume. 

In [2]:
# def extract_pdf_with_table_detection(pdf_path, output_csv_path):
#     """
#     Extract tabular data from PDF using improved pattern matching for all pages
#     """
#     all_data = []
    
#     with pdfplumber.open(pdf_path) as pdf:
#         for page_num, page in enumerate(pdf.pages, 1):
#             print(f"Processing page {page_num}...")
            
#             # Extract text from the page
#             text = page.extract_text()
            
#             if text:
#                 lines = text.split('\n')
                
#                 for line in lines:
#                     line = line.strip()
                    
#                     # Skip completely empty lines
#                     if not line:
#                         continue
                    
#                     # Skip obvious headers and section titles
#                     skip_patterns = [
#                         r'^PROGRAMS.*ACTIVITIES.*PROJECTS.*AMOUNT',
#                         r'^ORGANIZATIONAL OUTCOME',
#                         r'^Flood Management Program$',
#                         r'^Construction.*Maintenance.*Flood Mitigation',
#                         r'^National Capital Region$',
#                         r'^Central Office$',
#                         r'District Engineering Office\s*$',
#                         r'^Region I$',
#                         r'^DETAILS OF DPWH',
#                         r'^.*EXPENDITURE PROGRAM.*',
#                         r'^\d+ EXPENDITURE PROGRAM',
#                         r'^AMOUNT.*Php.*$'
#                     ]
                    
#                     should_skip = False
#                     for pattern in skip_patterns:
#                         if re.match(pattern, line, re.IGNORECASE):
#                             should_skip = True
#                             break
                    
#                     if should_skip:
#                         continue
                    
#                     # More flexible pattern to catch project lines with amounts
#                     # This handles various spacing and formatting issues
#                     amount_patterns = [
#                         r'^(.+?)\s+([0-9]{2,}(?:,\d{3})*)$',  # Standard pattern
#                         r'^(.+?)\s+([0-9,]+)$',               # Simple comma-separated numbers
#                         r'^(.+?)\s+(\d+)$'                    # Numbers without commas
#                     ]
                    
#                     matched = False
#                     for pattern in amount_patterns:
#                         match = re.match(pattern, line)
#                         if match:
#                             project_name = match.group(1).strip()
#                             amount_str = match.group(2).strip()
                            
#                             # Filter out lines that are clearly not project entries
#                             if (len(project_name) < 15 or  # Too short to be a real project
#                                 project_name.isdigit() or  # Just numbers
#                                 amount_str.isdigit() and len(amount_str) < 6):  # Amount too small
#                                 continue
                            
#                             # Additional filters for known non-project lines
#                             if any(keyword in project_name.upper() for keyword in 
#                                   ['AMOUNT', 'PROGRAMS', 'ACTIVITIES', 'PROJECTS', 'PAGE']):
#                                 continue
# #                            
#                             try:
#                                 # Clean and convert amount
#                                 amount = int(amount_str.replace(',', ''))
                                
#                                 # Only include amounts that make sense for construction projects
#                                 if amount >= 1000:  # Minimum threshold
#                                     all_data.append({
#                                         'Project_Name': project_name,
#                                         'Amount_PHP': amount,
#                                         'Page_Number': page_num
#                                     })
#                                     matched = True
#                                     break
#                             except ValueError:
#                                 continue
                    
#                     # Debug: Print lines that don't match any pattern (optional)
#                     # Uncomment the next two lines to see what's being skipped
#                     # if not matched and len(line) > 20:
#                     #     print(f"Skipped line on page {page_num}: {line}")
            
#             print(f"Found {len([d for d in all_data if d['Page_Number'] == page_num])} projects on page {page_num}")
    
#     # Create DataFrame and save to CSV
#     if all_data:
#         df = pd.DataFrame(all_data)
        
#         df.to_csv("page12.csv", index=False)
#         print(f"\nSuccessfully extracted {len(df)} unique records to {output_csv_path}")
        
#         # Display summary by page
#         page_summary = df.groupby('Page_Number').agg({
#             'Project_Name': 'count',
#             'Amount_PHP': 'sum'
#         }).rename(columns={'Project_Name': 'Project_Count'})
#         print("\nSummary by page:")
#         print(page_summary)
        
#         print(f"\nTotal projects: {len(df)}")
#         print(f"Total budget: PHP {df['Amount_PHP'].sum():,}")
        
#         return df
#     else:
#         print("No data extracted")
#         return None

# # Usage example
# if __name__ == "__main__":
#     # Replace with your actual file paths
#     pdf_file_path = "flood-control_Part12.pdf"
#     csv_output_path = "page12.csv"
    
#     # Extract data
#     df = extract_pdf_with_table_detection(pdf_file_path, csv_output_path)
    
#     # Display first few rows if data was extracted
#     if df is not None:
#         print("\nFirst 10 rows of extracted data:")
#         print(df.head(10))
#         print("\nLast 10 rows of extracted data:")
#         print(df.tail(10))

## Fuzzy matching

We use fuzzy matching to find similar entries from both the list of <b>DPWH flood control projects</b> with <b>data from DIME.</b>

- Preparation: Upload and read your files
- Use the strip function to remove invisible white spaces from entries. This is necessary to ensure you match the data for both documents since pandas read white spaces.
- Convert the Project Name columns to strings.

In [15]:
flood_2026= pd.read_excel('final_flood_control.xlsx', sheet_name='final')
completed = pd.read_excel('working-dime.xlsx')

In [35]:
flood_2026['project_name'] = flood_2026['project_name'].str.strip()
completed['dime_project_name'] = completed['dime_project_name'].str.strip()

In [36]:
completed.project_name = completed.dime_project_name.astype(str)
flood_2026.project_name = flood_2026.project_name.astype(str)

In [11]:
#pd.set_option('display.max_rows', None)
final_df = fpd.fuzzy_merge(flood_2026, completed,
                left_on=['project_name'],
                right_on=['dime_project_name'],
                ignore_case=True,
                ignore_nonalpha=True,
                ignore_nonlatin=True,
                threshold=0.9,
                #join='full-outer',
                keep='all')
final_df

,project_name,amount,page_no,pdf_file_no,dime_project_name,project_status,fund_source,dime_project_cost,start_date,completion_date
0,Construction of Flood Control Wall along the B...,95000000,3,1,Construction of Flood Control Wall along the B...,Completed,General Appropriations Act FY 2024,47975930.10,2024-02-27 00:00:00,2024-05-19 00:00:00
1,Construction of Flood Control Structure along ...,65000000,9,1,Construction of Flood Control Structure along ...,Not Yet Started,General Appropriations Act FY 2025,30000000.00,Not available,Not Available
2,"Construction of Flood Control Structure, Baran...",80000000,10,1,"Construction of Flood Control Structure, Baran...",Not Yet Started,General Appropriations Act FY 2025,83892000.00,Not available,Not Available
3,Construction of Flood Control Structure along ...,100000000,7,2,Construction of Flood Control Structure along ...,Completed,General Appropriations Act FY 2023,86848966.84,2023-02-27 00:00:00,2023-12-10 00:00:00
4,Construction of Flood Control Structure along ...,100000000,8,2,"Construction of Flood Control Structure, along...",Completed,General Appropriations Act FY 2022,94569655.86,2022-03-07 00:00:00,2022-11-23 00:00:00
...,...,...,...,...,...,...,...,...,...,...
81,Construction of Flood Control Structure (Revet...,50000000,8,11,Construction of Flood Control Structure (Revet...,Not Yet Started,General Appropriations Act FY 2025,40000000.00,Not available,Not Available
82,"Construction of Flood Control Structure, Ala R...",50000000,10,11,"Construction of Flood Control Structure, Ala R...",Completed,General Appropriations Act FY 2024,44954629.65,2024-02-15 00:00:00,2024-06-10 00:00:00
83,"Construction of Revetment along Surigao River,...",50000000,10,11,"Construction of Revetment along Surigao River,...",Completed,General Appropriations Act FY 2023,96472570.51,2023-04-18 00:00:00,2024-04-14 00:00:00
84,"Construction of Flood Control Structure, Ala R...",50000000,2,12,"Construction of Flood Control Structure, Ala R...",Completed,General Appropriations Act FY 2024,44954629.65,2024-02-15 00:00:00,2024-06-10 00:00:00


## Save the the file to CSV

Save the output of your fuzzy match showing duplicate entries to CSV. Remember that the output will include all projects with similar names from both DIME and the DPWH list. Ensure that you double check this as some projects may have similar names, but different locations or other characteristics. 

In [12]:
# final_df.to_csv('duplicates.csv', index=False)